In [1]:
from google.cloud import aiplatform
import os

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS']= "L:\\gcp-practice-0123-bd3866a21b22.json"

In [3]:
PROJECT_ID                 = 'gcp-practice-0123'
REGION                     = 'us-central1'
MACHINE_TYPE               = 'e2-standard-4'
BUCKET_URI                 = 'gcp-practice-0123-18jun2023/custom-trained-model/model/'

In [4]:
aiplatform.init(project        = PROJECT_ID,
                location       = REGION,
                staging_bucket = BUCKET_URI)

In [35]:
def create_endpoint(endpoint_display_name_in:str, 
                     project_in:str
                   ) -> str:
    print("\n\n***Creating Endpoint....")
    from google.cloud import aiplatform
    aiplatform.init(project=project_in)
    endpoint = aiplatform.Endpoint.create(project      = project_in,
                                          display_name = endpoint_display_name_in
                                         )
    print("***The End Point Project is : ",endpoint.project)
    print("***The Private End point resource name is : ",endpoint.resource_name)
    #vertex_endpoint.uri = endpoint.resource_name
    return endpoint.resource_name

In [36]:
def deploy_model(project_in:str,
                  model_display_name_in : str,
                  model_resource_nm_in:str,
                  endpoint_in : str
                ):
    print("\n\n***Deploying Model....")
    from google.cloud import aiplatform
    aiplatform.init(project=project_in)
    model_obj = aiplatform.Model(model_resource_nm_in)
    endpoin_obj = aiplatform.Endpoint(endpoint_in)
    deployed_model = model_obj.deploy(endpoint                    = endpoin_obj,
                                      deployed_model_display_name = model_display_name_in,
                                      min_replica_count           = 1,
                                      max_replica_count           = 1,
                                      traffic_split               = {"0":100}
                                     )

In [37]:
def register_model(project_in:str,
                    model_display_name_in : str,
                    model_gcs_path_in : str,
                    serving_container_in : str
                  ) -> str:
    print("\n\n***Registering Model....")
    from google.cloud import aiplatform
    aiplatform.init(project=project_in)
    my_model = aiplatform.Model.upload( project                     = project_in,
                                        display_name                = model_display_name_in,
                                        serving_container_image_uri = serving_container_in,
                                        artifact_uri                = model_gcs_path_in
                                      )
    print("***Model Resource Name is : ", my_model.resource_name)
    return my_model.resource_name

In [5]:
def batch_prediction( project_in:str,
                      model_display_name_in:str,
                      model_resource_nm_in:str,
                      bigquery_source_in:str,
                      bigquery_destination_prefix_in:str,
                      predictions_format_in:str,
                      machine_type_in:str
                    ):
    print("\n\nDoing Batch Prediction...")
    from google.cloud import aiplatform
    aiplatform.init(project=project_in)
    model_obj = aiplatform.Model(model_resource_nm_in)
    batch_predict_job = model_obj.batch_predict( job_display_name            = model_display_name_in,
                                                 bigquery_source             = bigquery_source_in,
                                                 bigquery_destination_prefix = bigquery_destination_prefix_in,
                                                 predictions_format          = predictions_format_in,
                                                 machine_type                = machine_type_in
                                               )

In [39]:
def loadBigQueryTableFromGCS(project_in:str,bucket_uri_in:str):
    from google.cloud import bigquery
    client = bigquery.Client(project=project_in,location='us-central1')
    job_config = bigquery.LoadJobConfig(
                                        schema=[
                                            bigquery.SchemaField("longitude",           "FLOAT"),
                                            bigquery.SchemaField("latitude",            "FLOAT"),
                                            bigquery.SchemaField("housing_median_age",  "FLOAT"),
                                            bigquery.SchemaField("total_rooms",         "FLOAT"),
                                            bigquery.SchemaField("total_bedrooms",      "FLOAT"),
                                            bigquery.SchemaField("population",          "FLOAT"),
                                            bigquery.SchemaField("households",          "FLOAT"),
                                            bigquery.SchemaField("median_income",       "FLOAT"),
                                            bigquery.SchemaField("median_house_value",  "FLOAT")
                                        ],
                                        skip_leading_rows = 1,
                                        # The source format defaults to CSV, so the line below is optional.
                                        source_format = bigquery.SourceFormat.CSV,
                                    )
    uri      = bucket_uri_in+'/california_housing_train.csv'
    table_id = 'gcp-practice-0123.dataset2.california_housing_test'
    load_job = client.load_table_from_uri(source_uris = uri,
                                          destination = table_id,
                                          job_config  = job_config)  # Make an API request.
    load_job.result()

In [40]:
def mlopsDeployPipeline(project_in:str,
                        endpoint_display_name_in:str,
                        model_gcs_path_in:str,
                        serving_container_in:str,
                        bigquery_source_in:str,
                        bigquery_destination_prefix_in:str,
                        predictions_format_in:str,
                        machine_type_in:str,
                        bucket_uri_in:str
                       ):
    #from create_endpoint import create_endpoint
    #from upload_model import register_model
    #from deploy_model import deploy_model
    #from batch_prediction import batch_prediction
    #from loadBigQueryTableFromGCS import loadBigQueryTableFromGCS
    
    endpoint_op = create_endpoint(endpoint_display_name_in,
                                  project_in
                                 )
    
    model_op = register_model(project_in,
                              endpoint_display_name_in,
                              model_gcs_path_in,
                              serving_container_in
                             )

    deploy_model(project_in,
                 endpoint_display_name_in,
                 model_op,
                 endpoint_op
                )
    
    loadBigQueryTableFromGCS(project_in,
                             bucket_uri_in
                            )
    
    batch_prediction(project_in,
                     endpoint_display_name_in,
                     model_op,
                     bigquery_source_in,
                     bigquery_destination_prefix_in,
                     predictions_format_in,
                     machine_type_in)

In [41]:
mlopsDeployPipeline(PROJECT_ID,
                    'mlops-pipeline-test',
                    'gs://gcp-practice-0123-18jun2023/custom-trained-model/model/aiplatform-custom-training-2023-01-22-18:07:17.227/model',
                    "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest",
                    "bq://gcp-practice-0123.dataset2.california_housing_test",
                    'bq://gcp-practice-0123.dataset2',
                    "bigquery",
                    "e2-standard-4",
                    "gs://gcp-practice-0123-18jun2023/custom-trained-model/training-data"
                   )



***Creating Endpoint....
Creating Endpoint
Create Endpoint backing LRO: projects/553411498874/locations/us-central1/endpoints/389900017348902912/operations/786692666933903360
Endpoint created. Resource name: projects/553411498874/locations/us-central1/endpoints/389900017348902912
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/553411498874/locations/us-central1/endpoints/389900017348902912')
***The End Point Project is :  gcp-practice-0123
***The Private End point resource name is :  projects/553411498874/locations/us-central1/endpoints/389900017348902912


***Registering Model....
Creating Model
Create Model backing LRO: projects/553411498874/locations/us-central1/models/295005566802067456/operations/1921599773031268352
Model created. Resource name: projects/553411498874/locations/us-central1/models/295005566802067456@1
To use this Model in another session:
model = aiplatform.Model('projects/553411498874/locations/us-central1/models/2950055668020674

In [43]:
pip install build

Note: you may need to restart the kernel to use updated packages.


In [2]:
import build

In [20]:
def loadBigQueryTableFromGCS(project_in:str,bucket_uri_in:str):
    from google.cloud import bigquery
    client = bigquery.Client(project=project_in,location='us-central1')
    job_config = bigquery.LoadJobConfig(
                                        schema=[
                                            bigquery.SchemaField("longitude",           "FLOAT"),
                                            bigquery.SchemaField("latitude",            "FLOAT"),
                                            bigquery.SchemaField("housing_median_age",  "FLOAT"),
                                            bigquery.SchemaField("total_rooms",         "FLOAT"),
                                            bigquery.SchemaField("total_bedrooms",      "FLOAT"),
                                            bigquery.SchemaField("population",          "FLOAT"),
                                            bigquery.SchemaField("households",          "FLOAT"),
                                            bigquery.SchemaField("median_income",       "FLOAT"),
                                            bigquery.SchemaField("median_house_value",  "FLOAT")
                                        ],
                                        skip_leading_rows = 1,
                                        # The source format defaults to CSV, so the line below is optional.
                                        source_format = bigquery.SourceFormat.CSV,
                                    )
    uri      = bucket_uri_in+'/california_housing_train.csv'
    table_id = 'gcp-practice-0123.dataset2.california_housing_test'
    load_job = client.load_table_from_uri(source_uris = uri,
                                          destination = table_id,
                                          job_config  = job_config)  # Make an API request.
    load_job.result()

In [21]:
loadBigQueryTableFromGCS(PROJECT_ID,"gs://gcp-practice-0123-18jun2023/custom-trained-model/training-data")

In [ ]:
loa

In [6]:
model = aiplatform.Model('projects/553411498874/locations/us-central1/models/847259471108374528@1')

In [25]:
model = aiplatform.Model('projects/553411498874/locations/us-central1/models/847259471108374528')

In [26]:
model

resource name: projects/553411498874/locations/us-central1/models/847259471108374528

In [30]:
model = aiplatform.Model(model_name='projects/553411498874/locations/us-central1/models/847259471108374528',
                         version=1
                        )

In [ ]:
model.batch_predict()

In [8]:
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID,location='us-central1')
client.

In [12]:
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID,location='us-central1')
job_config = bigquery.LoadJobConfig(
                                    schema=[
                                        bigquery.SchemaField("longitude",           "FLOAT"),
                                        bigquery.SchemaField("latitude",            "FLOAT"),
                                        bigquery.SchemaField("housing_median_age",  "FLOAT"),
                                        bigquery.SchemaField("total_rooms",         "FLOAT"),
                                        bigquery.SchemaField("total_bedrooms",      "FLOAT"),
                                        bigquery.SchemaField("population",          "FLOAT"),
                                        bigquery.SchemaField("households",          "FLOAT"),
                                        bigquery.SchemaField("median_income",       "FLOAT"),
                                        bigquery.SchemaField("median_house_value",  "FLOAT")
                                    ],
                                    skip_leading_rows = 1,
                                    # The source format defaults to CSV, so the line below is optional.
                                    source_format = bigquery.SourceFormat.CSV,
                                )
uri      = 'gs://gcp-practice-0123-18jun2023/custom-trained-model/training-data/california_housing_train.csv'
table_id = 'gcp-practice-0123.dataset2.california_housing_test'
load_job = client.load_table_from_uri(source_uris = "gs://gcp-practice-0123-18jun2023/custom-trained-model/training-data",
                                      destination = table_id,
                                      job_config  = job_config)  # Make an API request.

In [ ]:
load_job.allow_jagged_rows

In [15]:
model = aiplatform.Model('projects/553411498874/locations/us-central1/models/2607041025503395840@1')

In [ ]:
model_obj.batch_predict()

In [18]:
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID)
model_obj = aiplatform.Model('projects/553411498874/locations/us-central1/models/2607041025503395840@1')
batch_predict_job = model_obj.batch_predict( job_display_name            = 'model_display_name_in',
                                             bigquery_source             = 'bq://gcp-practice-0123.dataset2.california_housing_test',
                                             bigquery_destination_prefix = 'gcp-practice-0123.dataset2',
                                             predictions_format          = 'bigquery',
                                             machine_type                = "e2-standard-4"
                                           )

Creating BatchPredictionJob
BatchPredictionJob created. Resource name: projects/553411498874/locations/us-central1/batchPredictionJobs/5448048367231827968
To use this BatchPredictionJob in another session:
bpj = aiplatform.BatchPredictionJob('projects/553411498874/locations/us-central1/batchPredictionJobs/5448048367231827968')
View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/5448048367231827968?project=553411498874
BatchPredictionJob projects/553411498874/locations/us-central1/batchPredictionJobs/5448048367231827968 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/553411498874/locations/us-central1/batchPredictionJobs/5448048367231827968 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/553411498874/locations/us-central1/batchPredictionJobs/5448048367231827968 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/553411498874/locations/us-central1/batchPredictionJobs/

In [ ]:
batch_predict_job.